# A Trader Stabilize Fiat Value of Crypto Currency

In [1]:
TEST_RATIO = 1

In [2]:
SAMPLE_INTERVAL = 60
HAPPY_COOLING_TIME = 60
SAD_COOLING_TIME = 60 * 6
BUY_RATE = 15
BUY_AMOUNT = 0.04 / 4 / TEST_RATIO
MAX_SELL_COUNT = 128
MAX_USED_JPY = 400000 / 4
MIN_UNIT_CC_GAIN_JPY = 400
MIN_UNIT_CC_LOSS_JPY = 400
MAX_UNIT_CC_LOSS_JPY = 4000
MAX_LOSS_JPY = 4000 / 4
MAX_TOTAL_LOSS_JPY = 8000 / 4

In [3]:
from datetime import datetime
import json, sys, time
import python_bitbankcc

In [4]:
pub = python_bitbankcc.public()

## Simulated One

In [5]:
from unittest.mock import Mock
prv = Mock()

## Actual One

In [6]:
# import os
# prv = python_bitbankcc.private(
#     os.environ['BITBANK_API_KEY'],
#     os.environ['BITBANK_API_SECRET']
# )

## Start Trading

In [7]:
sample_number = 0
buy_count = 0
sell_count = 0
bought_unit_amount = 0
used_jpy = 0
please_buy_unit_amount = 0
please_sell_unit_amount = 0
cooling_time = HAPPY_COOLING_TIME
total_gained_jpy = 0

In [ ]:
with open('sample-history.txt', 'a', 1) as fsh, open('transaction-history.txt', 'a', 1) as fth:
    while True:
        now_ticker = pub.get_ticker('eth_jpy')
        now_sell_jpy = int(now_ticker.get('buy', '0'))
        now_buy_jpy = int(now_ticker.get('sell', '1'))
        # comment out to speed up
        fsh.write(f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n')

        bought_amount = BUY_AMOUNT * bought_unit_amount

        if sample_number % BUY_RATE == 0:
            if bought_unit_amount > 0 and now_sell_jpy < used_jpy/bought_amount - MAX_UNIT_CC_LOSS_JPY:
                please_sell_unit_amount = bought_unit_amount
                cooling_time = SAD_COOLING_TIME

            if now_sell_jpy * bought_amount - used_jpy < -MAX_LOSS_JPY:
                please_sell_unit_amount = bought_unit_amount
                cooling_time = SAD_COOLING_TIME

        if sample_number % BUY_RATE == 0 and bought_unit_amount > 0 and now_sell_jpy > used_jpy/bought_amount:
            please_sell_unit_amount = 1
            cooling_time = HAPPY_COOLING_TIME

        if bought_unit_amount > 0 and now_sell_jpy > used_jpy/bought_amount + MIN_UNIT_CC_GAIN_JPY \
                * (MAX_USED_JPY ** 2 - used_jpy ** 2) ** 0.5 / MAX_USED_JPY:
            sample_number = 0
            please_sell_unit_amount = (bought_unit_amount + 1) // 2
            cooling_time = HAPPY_COOLING_TIME

        if please_sell_unit_amount > 0:
            please_sell_amount = BUY_AMOUNT * please_sell_unit_amount
            prv.order(
                pair='eth_jpy', price=str(now_sell_jpy), amount=str(please_sell_amount), side='sell', order_type='market'
            )

            avg_jpy = used_jpy / bought_amount
            diff_jpy = now_sell_jpy - avg_jpy
            gained_jpy = diff_jpy * please_sell_amount
            total_gained_jpy += gained_jpy
            fth.write(
                f'SOLD {please_sell_amount} ETH for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY：BUY ETH for {now_buy_jpy} JPY\n'
                f'GAINED {gained_jpy} JPY: GAINED {total_gained_jpy} JPY in total at {datetime.now()}\n\n'
            )

            buy_count = 0
            sell_count += 1
            bought_unit_amount -= please_sell_unit_amount
            used_jpy *= bought_unit_amount / (bought_unit_amount + please_sell_unit_amount)
            please_buy_unit_amount = 0
            please_sell_unit_amount = 0

            if sell_count == MAX_SELL_COUNT or total_gained_jpy < -MAX_TOTAL_LOSS_JPY:
                sell_count = 0
                cooling_time = HAPPY_COOLING_TIME
                total_gained_jpy = 0
                break

            time.sleep(cooling_time/TEST_RATIO)
            cooling_time = HAPPY_COOLING_TIME
            continue

        if sample_number % BUY_RATE == 0 and (bought_unit_amount == 0 or now_buy_jpy < used_jpy/bought_amount) \
                and used_jpy + now_buy_jpy * BUY_AMOUNT <= MAX_USED_JPY:
            buy_count += 1
            please_buy_unit_amount = 1

        if bought_unit_amount > 0 and now_buy_jpy < used_jpy/bought_amount - MIN_UNIT_CC_LOSS_JPY \
                * (used_jpy * MAX_USED_JPY * 2 - used_jpy ** 2) ** 0.5 / MAX_USED_JPY \
                and used_jpy + now_buy_jpy * bought_amount <= MAX_USED_JPY:
            sample_number = 0
            buy_count += 1
            please_buy_unit_amount = bought_unit_amount

        if please_buy_unit_amount > 0:
            please_buy_amount = BUY_AMOUNT * please_buy_unit_amount
            prv.order(
                pair='eth_jpy', price=str(now_buy_jpy), amount=str(please_buy_amount), side='buy', order_type='market'
            )

            bought_unit_amount += please_buy_unit_amount
            used_jpy += now_buy_jpy * please_buy_amount
            please_buy_unit_amount = 0

            bought_amount = BUY_AMOUNT * bought_unit_amount
            avg_jpy = used_jpy / bought_amount
            diff_jpy = now_sell_jpy - avg_jpy
            fth.write(
                f'BOUGHT {please_buy_amount} ETH for {now_buy_jpy} JPY: SELL for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY: NO.{buy_count} at {datetime.now()}\n'
            )
        elif sample_number % BUY_RATE == 0:
            bought_amount = BUY_AMOUNT * bought_unit_amount
            avg_jpy = used_jpy / bought_amount
            diff_jpy = now_sell_jpy - avg_jpy
            fth.write(
                f'BUY for {now_buy_jpy} JPY: SELL for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY: at {datetime.now()}\n'
            )
            time.sleep(SAMPLE_INTERVAL / TEST_RATIO)
            continue

        time.sleep(SAMPLE_INTERVAL / TEST_RATIO)
        sample_number += 1